In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [2]:
df = pd.read_parquet("escooter_history_2022.parquet")



In [3]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)

In [4]:
df = pd.get_dummies(df)

In [27]:
df_1 = df[df["registered_customer"] == True].copy()

In [28]:
df_2 = df_1.groupby(pd.Grouper(key="datetime",freq="D")).agg(Anzahl_pro_Tag=("registered_customer","count"),temp=("temp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),workingday=("workingday","mean"),holiday=("holiday","mean"),Anzahl_registered=("registered_customer","sum"),clear_few_clouds=("weather_clear, few clouds","mean"),cloudy_mist=("weather_cloudy, mist","mean"),heacy_rain_thunderstorm_snow_icepallets=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),ligth_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()

In [29]:
df_2

,datetime,Anzahl_pro_Tag,temp,humidity,windspeed,workingday,holiday,Anzahl_registered,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm
0,2020-01-04,654,16.298440,80.010703,15.715434,0.0,0.0,654,0.316514,0.596330,0.0,0.087156
1,2020-01-05,670,14.419761,67.120896,15.811900,0.0,0.0,670,0.210448,0.510448,0.0,0.279104
2,2020-01-06,1229,8.372807,40.632221,14.603443,1.0,0.0,1229,1.000000,0.000000,0.0,0.000000
3,2020-01-07,1454,8.972063,56.616919,12.173591,1.0,0.0,1454,0.974553,0.025447,0.0,0.000000
4,2020-01-08,1518,9.609341,38.383399,14.038239,1.0,0.0,1518,1.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
726,2021-12-30,4024,10.463559,59.591700,22.123152,1.0,0.0,4024,0.543986,0.450298,0.0,0.005716
727,2021-12-31,7629,10.397415,58.792896,10.275610,1.0,0.0,7629,0.328352,0.671648,0.0,0.000000
728,2022-01-01,5293,10.553724,76.329681,8.286138,0.0,0.0,5293,0.213678,0.625165,0.0,0.161156
729,2022-01-02,4392,10.754472,46.234517,24.047083,0.0,0.0,4392,0.765255,0.234745,0.0,0.000000


In [20]:
df_2.dtypes

datetime             datetime64[ns]
Anzahl_pro_Tag                int64
temp                        float64
humidity                    float64
windspeed                   float64
workingday                  float64
holiday                     float64
Anzahl_registered             Int64
Anteil_registered           Float64
weekend                        bool
date_delta                    int64
dtype: object

In [30]:
df_2["workingday"] = df_2["workingday"].round()

In [31]:

df_2["holiday"] = df_2["holiday"].round() 

In [10]:
df_2["Anteil_registered"] = df_2["Anzahl_registered"]/df_2["Anzahl_pro_Tag"]

In [32]:
df_2["weekend"] = df_2["datetime"].dt.weekday > 4

In [33]:
df_2['date_delta'] = (df_2['datetime'] - df_2['datetime'].min())  
df_2["date_delta"] = df_2.date_delta.dt.days

In [13]:
df_2.drop("Anzahl_registered",axis=1)

,datetime,Anzahl_pro_Tag,temp,humidity,windspeed,workingday,holiday,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm,Anteil_registered,weekend,date_delta
0,2020-01-04,331,16.868218,79.021148,17.235598,0.0,0.0,0.296073,0.658610,0.0,0.045317,0.0,True,0
1,2020-01-05,131,14.390687,67.244275,16.198554,0.0,0.0,0.183206,0.572519,0.0,0.244275,0.0,True,1
2,2020-01-06,120,9.129333,36.758333,15.966826,1.0,0.0,1.000000,0.000000,0.0,0.000000,0.0,False,2
3,2020-01-07,108,10.120926,53.638889,14.749616,1.0,0.0,0.953704,0.046296,0.0,0.000000,0.0,False,3
4,2020-01-08,82,10.390000,35.390244,15.500532,1.0,0.0,1.000000,0.000000,0.0,0.000000,0.0,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2021-12-30,207,10.707536,57.149758,24.556083,1.0,0.0,0.468599,0.531401,0.0,0.000000,0.0,False,726
727,2021-12-31,515,10.741204,56.588350,11.313828,1.0,0.0,0.326214,0.673786,0.0,0.000000,0.0,False,727
728,2022-01-01,639,10.535524,76.259781,7.836127,0.0,0.0,0.187793,0.674491,0.0,0.137715,0.0,True,728
729,2022-01-02,790,10.913266,46.292405,23.570295,0.0,0.0,0.739241,0.260759,0.0,0.000000,0.0,True,729


In [34]:
df_2["log_temp"] = np.log(df_2.temp)

In [34]:
df_2["log_atemp"] = np.log(df_2.atemp)

AttributeError: 'DataFrame' object has no attribute 'atemp'

In [ ]:
pd.plotting.scatter_matrix(df_2['temp', 'atemp',
       'humidity', 'windspeed',"Anzahl_pro_Tag"])

In [35]:
#trainings-Test-Split
features = ["date_delta", 'temp',"log_temp", 'humidity','windspeed',"workingday","holiday","weekend","clear_few_clouds","cloudy_mist","heacy_rain_thunderstorm_snow_icepallets","ligth_snow_rain_thunderstorm"] # Liste von Features
X = df_2[features]
y = df_2.Anzahl_pro_Tag

In [36]:
X

,date_delta,temp,log_temp,humidity,windspeed,workingday,holiday,weekend,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm
0,0,16.298440,2.791069,80.010703,15.715434,0.0,0.0,True,0.316514,0.596330,0.0,0.087156
1,1,14.419761,2.668600,67.120896,15.811900,0.0,0.0,True,0.210448,0.510448,0.0,0.279104
2,2,8.372807,2.124989,40.632221,14.603443,1.0,0.0,False,1.000000,0.000000,0.0,0.000000
3,3,8.972063,2.194116,56.616919,12.173591,1.0,0.0,False,0.974553,0.025447,0.0,0.000000
4,4,9.609341,2.262736,38.383399,14.038239,1.0,0.0,False,1.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
726,726,10.463559,2.347899,59.591700,22.123152,1.0,0.0,False,0.543986,0.450298,0.0,0.005716
727,727,10.397415,2.341557,58.792896,10.275610,1.0,0.0,False,0.328352,0.671648,0.0,0.000000
728,728,10.553724,2.356479,76.329681,8.286138,0.0,0.0,True,0.213678,0.625165,0.0,0.161156
729,729,10.754472,2.375322,46.234517,24.047083,0.0,0.0,True,0.765255,0.234745,0.0,0.000000


In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler(with_mean=False) # with_mean=False: dont rescale the means to being all zero
Xs = pd.DataFrame(scaler.fit_transform(X[X.columns]))
Xs.columns = X.columns

In [38]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [39]:
regr = linear_model.LinearRegression()
regr.fit(Xs, y)
pred = regr.predict(Xs)
print(Xs.columns.values)
print('Coefficients: \n', regr.coef_)
print('Intercept: \n', regr.intercept_)
print('On Training set:')
print("Mean squared error: %.2f" % mean_squared_error(y, pred))
print("Variance of target variable: %.2f" % np.var(y))
print('Variance explained: %.2f' % r2_score(y, pred))

['date_delta' 'temp' 'log_temp' 'humidity' 'windspeed' 'workingday'
 'holiday' 'weekend' 'clear_few_clouds' 'cloudy_mist'
 'heacy_rain_thunderstorm_snow_icepallets' 'ligth_snow_rain_thunderstorm']
Coefficients: 
 [ 1.10199828e+03  9.26401442e+01  5.57458745e+02 -1.56179164e+02
 -1.40685278e+02 -1.83839818e+15 -6.60522187e+14 -1.78927502e+15
 -2.54278583e+16 -2.07728555e+16 -2.79245242e+14 -1.13988269e+16]
Intercept: 
 8.099750892218894e+16
On Training set:
Mean squared error: 1221744.97
Variance of target variable: 3622261.41
Variance explained: 0.66


In [40]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [42]:
from sklearn.preprocessing import StandardScaler
#Standardisierung

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [43]:
#Training

model_lr = linear_model.LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

In [44]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [45]:
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

In [46]:
print(r2_test)

0.639370050765766


In [47]:
print(mse_test)

1456490.2639189712


In [48]:
print(mae_test)

860.4939873770641


problem cannot turn datetime into float